In [52]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
import psycopg2
from datetime import datetime
import locale
import sys

In [2]:
# Estabelecimento de conexão com o banco de dados
# para extração das informações das tabelas necessárias

connection = psycopg2.connect(
        user="cat",
        password="5pM2h0MBQu9JHkxHud2A",
        host="177.11.49.194",
        port="3361",
        database="cat"
    )

In [3]:
# Definição da loja a ser utilizada

loja = 14
periodo = 0

In [4]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha1 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx')
planilha2 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx')

planilha = pd.concat([planilha1, planilha2])
planilha = planilha.reset_index()
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,COD_ITEM,CNPJ EMITENTE,tipo,vBCST,FONTE
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,3,5.0,...,0.0,NaN,0.0,0.0,0.0,11,6.161357e+12,NaN,0.000000,saida_nfe
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,2,NaN,...,0.0,NaN,0.0,0.0,0.0,11,6.161357e+12,NaN,0.000000,saida_nfe
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,3,119.0,...,0.0,0.0,0.0,0.0,0.0,18,NaN,grupo,0.000000,dfe
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,4,NaN,...,0.0,0.0,0.0,0.0,0.0,18,NaN,grupo,0.000000,dfe
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,2,NaN,...,0.0,0.0,0.0,0.0,0.0,18,NaN,grupo,0.000000,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,99325,2.123509e+13,outros2,12.376856,entrada_sp
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,6,NaN,...,0.0,0.0,0.0,0.0,0.0,99325,2.123509e+13,outros2,74.261136,entrada_sp
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,99325,NaN,outros2,0.000000,dfe
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,1,NaN,...,0.0,0.0,0.0,0.0,0.0,99325,NaN,outros2,0.000000,dfe


In [5]:
planilha['VLR_RESSARCIMENTO'].sum() / planilha[planilha['CFOP'] == '5405']['VALOR'].sum()

C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3656415162.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  planilha['VLR_RESSARCIMENTO'].sum() / planilha[planilha['CFOP'] == '5405']['VALOR'].sum()


inf

In [6]:
tabela_2 = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='latin-1')
tabela_2 = tabela_2.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                           ascending=[True, True, True, False])
tabela_2

C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3625351105.py:1: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tabela_2 = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='latin-1')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,tipo,COD_ITEM,IND_OPER,SUB_TIPO,...,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base CÃ¡lculo ICMS SubstituiÃ§Ã£o TributÃ¡ria,vBCST,FONTE
1251041,1251041,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,NaN,11,1,-1,...,BATOM COR 02 VULT,7898417961020,UNI,18040000.0,2806100.0,6.161357e+12,1.032898e+13,NaN,0.000000,saida_nfe
1251059,1251059,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,NaN,11,1,-1,...,BATOM COR 02 VULT,7898417961020,UNI,18040000.0,2806100.0,6.161357e+12,1.032898e+13,NaN,0.000000,saida_nfe
596183,596183,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,grupo,18,1,1,...,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
596177,596177,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,grupo,18,1,1,...,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
596184,596184,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,grupo,18,1,1,...,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145597,145597,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,outros2,99325,0,1,...,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090.0,NaN,2.123509e+13,6.161357e+12,NaN,12.376856,entrada_sp
145157,145157,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,outros2,99325,0,1,...,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090.0,NaN,2.123509e+13,6.161357e+12,NaN,74.261136,entrada_sp
887116,887116,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,outros2,99325,1,1,...,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090.0,NaN,NaN,NaN,NaN,0.000000,dfe
887128,887128,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,outros2,99325,1,1,...,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090.0,NaN,NaN,NaN,NaN,0.000000,dfe


In [7]:
planilha = planilha.merge(tabela_2[['CHV_DOC', 'NUM_ITEM', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M', 'CEST',
                'CNPJ DESTINATARIO']],
            on=['CHV_DOC', 'NUM_ITEM'], how='left')

In [8]:
planilha.head()

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,3,5.0,...,6.161357e+12,NaN,0.0,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000.0,2806100.0,1.032898e+13
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,2,NaN,...,6.161357e+12,NaN,0.0,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000.0,2806100.0,1.032898e+13
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,3,119.0,...,NaN,grupo,0.0,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,4,NaN,...,NaN,grupo,0.0,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,2,NaN,...,NaN,grupo,0.0,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099.0,NaN,NaN


In [9]:
planilha['cod novo'] = planilha['cod novo'].astype(str)

In [10]:
planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")

In [11]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,21235094000158,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090,nan,6.161357e+12
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,21235094000158,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090,nan,6.161357e+12
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN


In [12]:
planilha = planilha.drop_duplicates()
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,21235094000158,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090,nan,6.161357e+12
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,21235094000158,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437.0,UN,33072090,nan,6.161357e+12
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN


In [13]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,6161357000804,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,NaN,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,21235094000158,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,nan,6.161357e+12
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,21235094000158,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,nan,6.161357e+12
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,NaN,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,nan,NaN


In [14]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['012018', '112016', '122016', '032017', '042017', '052017',
       '062017', '072017', '092017', '102017', '122017', '022018',
       '032018', '042018', '012017', '022017', '112017', '082017',
       '102016'], dtype=object)

In [15]:
planilha.replace('nan', np.nan, inplace=True)

In [16]:
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,082017
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,012018


In [17]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0          2806100
1          2806100
2              NaN
3              NaN
4              NaN
            ...   
1252960        NaN
1252961        NaN
1252962        NaN
1252963        NaN
1252964    2803700
Name: CEST, Length: 1252965, dtype: object

In [18]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,082017
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,012018


In [19]:
planilha.columns

Index(['index', 'CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER',
       'SUB_TIPO', 'QTD_CAT', 'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent',
       'ICMS_TOT_0', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit',
       'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
       'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VLR_CONF_0',
       'VLR_CONFR_UNIT', 'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO',
       'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL',
       'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
       'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'ICMS_TOT', 'COD_ITEM', 'CNPJ EMITENTE', 'tipo',
       'vBCST', 'FONTE', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M',
       'CEST', 'CNPJ DESTINATARIO', 'Data'],
      dtype='object')

In [20]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data


In [21]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35180106161357000804550020000178621133143055,2018-01-27,5411,1,11,1,-1,"3,000",5.0,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
1,1,35180106161357000804550020000178631133143125,2018-01-27,5411,1,11,1,-1,"2,000",NaN,...,NaN,0.000000,saida_nfe,BATOM COR 02 VULT,7898417961020,UNI,18040000,2806100,1.032898e+13,012018
2,2,35161106161357000804590000459520151204835367,2016-11-03,5405,2,18,1,1,"3,000",119.0,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
3,3,35161106161357000804590000162330432911476882,2016-11-04,5405,1,18,1,1,"4,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
4,4,35161106161357000804590000459520153874426811,2016-11-04,5405,1,18,1,1,"2,000",NaN,...,grupo,0.000000,dfe,DEPAKENE 250MG 25.S ABB C1,NaN,UN,30049099,NaN,NaN,112016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,644642,35170721235094000158550010003023611003319830,2017-07-20,1403,62,941848,0,1,"1,000",NaN,...,outros2,12.376856,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252961,644643,35170721235094000158550010003023141003319368,2017-07-21,1403,10,941848,0,1,"6,000",NaN,...,outros2,74.261136,entrada_sp,LT MONANGE 400ML FLOR LAVANDA,7896094995437,UN,33072090,NaN,6.161357e+12,072017
1252962,644644,35170806161357000804590000461461240937130891,2017-08-04,5405,1,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,082017
1252963,644645,35180106161357000804590000033321373300151619,2018-01-15,5405,6,941848,1,1,"1,000",NaN,...,outros2,0.000000,dfe,LT MONANGE 400ML FLOR LAVANDA,NaN,UN,33072090,NaN,NaN,012018


In [22]:
list(planilha['CODIGO_BARRA'].unique())

['7898417961020 ',
 nan,
 '7891158000942',
 '7891158000942 ',
 '7898417961136 ',
 '7898417961174 ',
 '8901580008900',
 '7891158000898 ',
 '7898417961242 ',
 '7891158000966',
 '7891158000966 ',
 '7898417961327 ',
 '7898417961471 ',
 '7898029556829 ',
 '7898029556829',
 '7899852000442 ',
 '7899852000459 ',
 '7891158003738',
 '7895296014014',
 '7898417964625 ',
 '7898040322540',
 '8902220109506',
 '7897322701912',
 '7898417962683 ',
 '7897322701516',
 '7896317907292',
 '7896317907292 ',
 '7896261005297',
 '7898417965523 ',
 '7898417965530 ',
 '7898417965592 ',
 '7898490059683 ',
 '7898490059690 ',
 '7899026443754',
 '7891268102482',
 '7898587779890',
 '7899026443778',
 '7896658002113',
 '7896010000009',
 '7896658001116',
 '7891721003042',
 '7898587779913',
 '7896658001178',
 '7896641803178',
 '7896658000805',
 '7896658000805 ',
 '7899026443822',
 '7896658007255',
 '7896658007255 ',
 '7891317001568',
 '7891317001568 ',
 '7891317001650',
 '7891317001636',
 '7891317001636 ',
 '7897595619518'

In [23]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'cod novo', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT', 'FONTE']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,cod novo,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT,FONTE
0,1100,35180106161357000804550020000178621133143055,2018-01-27,27012018,1,1,11,5411,"3,000",0.0,NaN,NaN,saida_nfe
1,1100,35180106161357000804550020000178631133143125,2018-01-27,27012018,1,1,11,5411,"2,000",0.0,NaN,NaN,saida_nfe
2,1100,35161106161357000804590000459520151204835367,2016-11-03,03112016,2,1,18,5405,"3,000",NaN,10.4004,1,dfe
3,1100,35161106161357000804590000162330432911476882,2016-11-04,04112016,1,1,18,5405,"4,000",NaN,13.8672,1,dfe
4,1100,35161106161357000804590000459520153874426811,2016-11-04,04112016,1,1,18,5405,"2,000",NaN,6.1524,1,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,1100,35170721235094000158550010003023611003319830,2017-07-20,20072017,62,0,941848,1403,"1,000",0.0,NaN,NaN,entrada_sp
1252961,1100,35170721235094000158550010003023141003319368,2017-07-21,21072017,10,0,941848,1403,"6,000",0.0,NaN,NaN,entrada_sp
1252962,1100,35170806161357000804590000461461240937130891,2017-08-04,04082017,1,1,941848,5405,"1,000",NaN,NaN,0,dfe
1252963,1100,35180106161357000804590000033321373300151619,2018-01-15,15012018,6,1,941848,5405,"1,000",NaN,NaN,0,dfe


In [24]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0          012018
1          012018
2          112016
3          112016
4          112016
            ...  
1252960    072017
1252961    072017
1252962    082017
1252963    012018
1252964    112017
Name: Data, Length: 1252965, dtype: object

In [25]:
df_1100.dtypes

COD_REG            object
CHV_DOC            object
Data               object
DATA               object
NUM_ITEM            int64
IND_OPER            int64
cod novo           object
CFOP               object
QTD_FIN            object
ICMS_TOT_PCAT     float64
VLR_CONFR_PCAT    float64
COD_LEGAL_PCAT     object
FONTE              object
dtype: object

In [26]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,cod novo,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT,FONTE
0,1100,35180106161357000804550020000178621133143055,012018,27012018,1,1,11,5411,"3,000",0.0,NaN,NaN,saida_nfe
1,1100,35180106161357000804550020000178631133143125,012018,27012018,1,1,11,5411,"2,000",0.0,NaN,NaN,saida_nfe
2,1100,35161106161357000804590000459520151204835367,112016,03112016,2,1,18,5405,"3,000",NaN,10.4004,1,dfe
3,1100,35161106161357000804590000162330432911476882,112016,04112016,1,1,18,5405,"4,000",NaN,13.8672,1,dfe
4,1100,35161106161357000804590000459520153874426811,112016,04112016,1,1,18,5405,"2,000",NaN,6.1524,1,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252960,1100,35170721235094000158550010003023611003319830,072017,20072017,62,0,941848,1403,"1,000",0.0,NaN,NaN,entrada_sp
1252961,1100,35170721235094000158550010003023141003319368,072017,21072017,10,0,941848,1403,"6,000",0.0,NaN,NaN,entrada_sp
1252962,1100,35170806161357000804590000461461240937130891,082017,04082017,1,1,941848,5405,"1,000",NaN,NaN,0,dfe
1252963,1100,35180106161357000804590000033321373300151619,012018,15012018,6,1,941848,5405,"1,000",NaN,NaN,0,dfe


In [27]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\1050_loja_{loja}_p{periodo}_v1.xlsx', dtype=str)

In [28]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF']]
arquivo_1050

,Unnamed: 0,cod novo,REF,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,11,201801,5,0,0,0,012018
1,1,18,201611,119,464.1009096199763,23,89.70017580890297,112016
2,2,18,201612,23,89.70017580890297,0,0,122016
3,3,18,201703,0,0,10,42.30904917069849,032017
4,4,18,201704,10,42.30904917069849,28,121.7701376779558,042017
...,...,...,...,...,...,...,...,...
153426,153426,941848,201706,7,0,6,0,062017
153427,153427,941848,201707,6,0,12,0,072017
153428,153428,941848,201708,12,0,11,0,082017
153429,153429,941848,201801,11,0,10,0,012018


In [29]:
for date in planilha['Data'].unique():
    print(date)

012018
112016
122016
032017
042017
052017
062017
072017
092017
102017
122017
022018
032018
042018
012017
022017
112017
082017
102016


In [30]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='latin-1') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [31]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [55]:
df_0150 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0150', dtype=str)
df_0150 = df_0150[['LOJA', 'REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
df_0150

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,Loja 19,0150,20642,Aussel Com De Urnas Funerarias E Serv Lt,1058,96628573000133,NaN,3549904,NaN,20161001
1,Loja 19,0150,20642,Aussel Com De Urnas Funerarias E Serv Lt,01058,96628573000133,NaN,3549904,NaN,20161201
2,Loja 54,0150,6155,PENITENCIARIA FEMININA II DE TREMEMBE,1058,96291141015020,NaN,3554805,NaN,20161001
3,Loja 54,0150,6155,PENITENCIARIA FEMININA II DE TREMEMBE,1058,96291141015020,NaN,3554805,NaN,20161101
4,Loja 54,0150,CLI751708509,Penitenciaria feminina ii de tremembe,1058,96291141015020,NaN,3554805,NaN,20170501
...,...,...,...,...,...,...,...,...,...,...
161414,Loja 26,0150,FOR000346063,FRIGELAR COMERCIO E INDUSTRIA LTDA,1058,92660406002758,NaN,3205002,083066411,20170501
161415,Loja 43,0150,FOR000346063,FRIGELAR COMERCIO E INDUSTRIA LTDA,1058,92660406002758,NaN,3205002,083066411,20170501
161416,Loja 114,0150,FOR751899296,GEOVANE ALVES RODRIGUES ME,1058,06178139000106,NaN,2601102,032373970,20180301
161417,Loja 13,0150,FOR000360815,M M R IND & COM DE MAQ LTDA,1058,08148443000136,NaN,4304408,0220068801,20170801


In [56]:
df_0150 = df_0150[df_0150['LOJA'] == f'Loja {loja}']
if df_0150.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].str.lstrip('\n')
df_0150['COD_PART'] = df_0150['COD_PART'].str.replace(r'^(FOR|CLI)', '', regex=True)
df_0150

C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\1251421117.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\1251421117.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\1251421117.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
214,Loja 14,0150,OPE000000001,VISA,1058,88888888000000,NaN,3549904,NaN,012017
533,Loja 14,0150,751796581,Consumidor final,1058,61861621000154,NaN,3550308,NaN,102017
535,Loja 14,0150,000351948,Assistencia social dom jose gaspar,1058,61734232000168,NaN,3550308,NaN,062017
588,Loja 14,0150,000247215,Consumidor final,1058,60194990000682,NaN,3549904,NaN,022017
698,Loja 14,0150,000169080,Consumidor final,1058,53023560000128,NaN,3550308,NaN,012018
...,...,...,...,...,...,...,...,...,...,...
161141,Loja 14,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,3550308,105953178119,072017
161142,Loja 14,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,3550308,105953178119,082017
161143,Loja 14,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,3550308,105953178119,092017
161144,Loja 14,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,3550308,105953178119,022018


In [57]:
df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN', 'DATA']]
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
214,0150,OPE000000001,VISA,1058,88888888000000,NaN,NaN,3549904,012017
533,0150,751796581,Consumidor final,1058,61861621000154,NaN,NaN,3550308,102017
535,0150,000351948,Assistencia social dom jose gaspar,1058,61734232000168,NaN,NaN,3550308,062017
588,0150,000247215,Consumidor final,1058,60194990000682,NaN,NaN,3549904,022017
698,0150,000169080,Consumidor final,1058,53023560000128,NaN,NaN,3550308,012018
...,...,...,...,...,...,...,...,...,...
161141,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,105953178119,3550308,072017
161142,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,105953178119,3550308,082017
161143,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,105953178119,3550308,092017
161144,0150,000002529,Donke Materiais Para Construção Ltda - Epp,1058,48050330000118,NaN,105953178119,3550308,022018


In [58]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['DATA'].unique()

# Iterate through unique 'PERIODO' values
for unique_periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['DATA'] == unique_periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = pd.concat([df_0150_final, filtered_df])

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,OPE000000001,VISA,1058,88888888000000,NaN,NaN,3549904,012017
1,0150,000161899,Associacao Beneficente Irma Ideofranca,1058,02537887000187,NaN,NaN,3550308,012017
2,0150,000156517,Prevent senior private op. de saude ltda,1058,00461479004401,NaN,NaN,3550308,012017
3,0150,000156510,Prevent senior private op.de saude ltda,1058,00461479004150,NaN,NaN,3550308,012017
4,0150,000156522,Centro de atencao intregada a saude mental,1058,62779145000947,NaN,NaN,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [59]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['IE'].notnull()]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
5,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,114031767115,3550308,012017
6,0150,000000658,Mc drogaria ltda,1058,18383936000880,NaN,141570363110,3550308,012017
7,0150,000002589,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,012017
8,0150,000156500,Prevent senior private op.de saude ltda,1058,00461479001054,NaN,145389352117,3550308,012017
9,0150,000156521,Prevent senior op. de saude ltda,1058,00461479001305,NaN,145389361118,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [60]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = list(df_0150_final['DATA'].unique())
for date in dates:
    for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])

if df_cnpj_repetido.shape[0] > 0:
    print('Existe repetição de CNPJ. Favor verificar')
    print(f'O CNPJ repetido é: {list(df_cnpj_repetido["CNPJ"].unique())}')
    print(f'A data em que ocorreu a repetição é {list(df_cnpj_repetido["DATA"].unique())}')
    sys.exit()
            
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [61]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = list(df_0150_final['DATA'].unique())
for date in dates:
    for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])

if df_ie_repetido.shape[0] > 0:
    print('Existe repetição de Inscrição Estadual. Favor verificar')
    print(f'A IE repetido é: {list(df_ie_repetido["IE"].unique())}')
    print(f'A data em que ocorreu a repetição é {list(df_ie_repetido["DATA"].unique())}')
    sys.exit()
    
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [62]:
for cod_part in list(df_0150_final['COD_PART'].unique()):
    for date in df_0150_final['DATA']:
        if df_0150_final[(df_0150_final['COD_PART'] == cod_part) & (df_0150_final['DATA'] == date)].shape[0] > 1:
            mensagem = 'ATENÇÃO. EXISTE CODIGO DE PARTICIPANTE REPETIDO. FAVOR CHECAR'
            print(mensagem)
            print(f'O codigo repetido é igual a {cod_part}')
            print(f'A data da repetição é igual a {date}')
            sys.exit()

In [63]:
# Iterate through each file in the folder
df_0000 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000

,LOJA,REG,DATA,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,Loja 1,0000,102016,Master Formula Farmacia de Manipulação LTDA,71605265039297,701237302110,3555406,01,00
1,Loja 1,0000,112016,Master Formula Farmacia de Manipulação LTDA,71605265039297,701237302110,3555406,01,00
2,Loja 1,0000,122016,Master Formula Farmacia de Manipulação LTDA,71605265039297,701237302110,3555406,01,00
3,Loja 1,0000,012017,Master Formula Farmacia de Manipulação LTDA,71605265039297,701237302110,3555406,01,00
4,Loja 1,0000,022017,Master Formula Farmacia de Manipulação LTDA,71605265039297,701237302110,3555406,01,00
...,...,...,...,...,...,...,...,...,...
3210,Loja 99,0000,122017,Master Formula Farmacia de Manipulação LTDA,71605265042751,138160414119,3550308,01,00
3211,Loja 99,0000,012018,Master Formula Farmacia de Manipulação LTDA,71605265042751,138160414119,3550308,01,00
3212,Loja 99,0000,022018,Master Formula Farmacia de Manipulação LTDA,71605265042751,138160414119,3550308,01,00
3213,Loja 99,0000,032018,Master Formula Farmacia de Manipulação LTDA,71605265042751,138160414119,3550308,01,00


In [64]:
for date in planilha['Data'].unique():
    print(date)

012018
112016
122016
032017
042017
052017
062017
072017
092017
102017
122017
022018
032018
042018
012017
022017
112017
082017
102016


In [65]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,OPE000000001,VISA,1058,88888888000000,NaN,NaN,3549904,012017
1,0150,000161899,Associacao Beneficente Irma Ideofranca,1058,02537887000187,NaN,NaN,3550308,012017
2,0150,000156517,Prevent senior private op. de saude ltda,1058,00461479004401,NaN,NaN,3550308,012017
3,0150,000156510,Prevent senior private op.de saude ltda,1058,00461479004150,NaN,NaN,3550308,012017
4,0150,000156522,Centro de atencao intregada a saude mental,1058,62779145000947,NaN,NaN,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [66]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    if not non_null_ie.empty:
        group['IE'] = non_null_ie.iloc[0]
    elif group['IE'].isnull().all():
        group['IE'] = np.nan
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)

df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,OPE000000001,VISA,1058,88888888000000,NaN,NaN,3549904,012017
1,0150,000161899,Associacao Beneficente Irma Ideofranca,1058,02537887000187,NaN,NaN,3550308,012017
2,0150,000156517,Prevent senior private op. de saude ltda,1058,00461479004401,NaN,NaN,3550308,012017
3,0150,000156510,Prevent senior private op.de saude ltda,1058,00461479004150,NaN,NaN,3550308,012017
4,0150,000156522,Centro de atencao intregada a saude mental,1058,62779145000947,NaN,NaN,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [67]:
df_0150_final_new = df_0150_final_new[df_0150_final_new['IE'].notnull()]
df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
5,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,114031767115,3550308,012017
6,0150,000000658,Mc drogaria ltda,1058,18383936000880,NaN,141570363110,3550308,012017
7,0150,000002589,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,012017
8,0150,000156500,Prevent senior private op.de saude ltda,1058,00461479001054,NaN,145389352117,3550308,012017
9,0150,000156521,Prevent senior op. de saude ltda,1058,00461479001305,NaN,145389361118,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [68]:
df_0150_final_new.loc[(df_0150_final_new['CNPJ'].notnull()) & (df_0150_final_new['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [69]:
df_0150_final_new['CNPJ'] = df_0150_final_new['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
df_0150_final_new

C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3208294570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150_final_new['CNPJ'] = df_0150_final_new['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
5,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,114031767115,3550308,012017
6,0150,000000658,Mc drogaria ltda,1058,18383936000880,NaN,141570363110,3550308,012017
7,0150,000002589,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,012017
8,0150,000156500,Prevent senior private op.de saude ltda,1058,00461479001054,NaN,145389352117,3550308,012017
9,0150,000156521,Prevent senior op. de saude ltda,1058,00461479001305,NaN,145389361118,3550308,012017
...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018


In [70]:
# Função para verificar o dígito verificador de um CNPJ
def verifica_digito_verificador(cnpj):
    cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
    if len(cnpj) != 14:
        return False  # CNPJ deve ter 14 dígitos

    # Calcula o primeiro dígito verificador
    soma = 0
    peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    for i in range(12):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador1 = 0
    else:
        digito_verificador1 = 11 - resto

    # Calcula o segundo dígito verificador
    soma = 0
    peso.insert(0, 6)
    for i in range(13):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador2 = 0
    else:
        digito_verificador2 = 11 - resto

    # Verifica se os dígitos verificadores são iguais aos dígitos originais
    return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

# Exemplo de DataFrame
df = df_0150_final_new.copy()

# Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)

# Exibe o DataFrame com a nova coluna
df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA,Dígito_Verificador_Válido
5,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,NaN,114031767115,3550308,012017,True
6,0150,000000658,Mc drogaria ltda,1058,18383936000880,NaN,141570363110,3550308,012017,True
7,0150,000002589,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003189,NaN,142004487112,3550308,012017,True
8,0150,000156500,Prevent senior private op.de saude ltda,1058,00461479001054,NaN,145389352117,3550308,012017,True
9,0150,000156521,Prevent senior op. de saude ltda,1058,00461479001305,NaN,145389361118,3550308,012017,True
...,...,...,...,...,...,...,...,...,...,...
767,0150,000000314,Conde Neto & Cia Ltda,1058,06161357001371,NaN,147552748114,3550308,042018,True
768,0150,000156512,Prevent Senior Private Operadora de Saude Ltda,1058,00461479001992,NaN,145503067113,3550308,042018,True
769,0150,000156514,Prevent Senior Private Operadora de Saude Ltda,1058,00461479003006,NaN,145389380113,3550308,042018,True
770,0150,000002528,Associacao Brasileira do Comercio Farmac,1058,53375317000179,NaN,142745654111,3550308,042018,True


In [71]:
df['Dígito_Verificador_Válido'].unique()

array([ True])

In [72]:
if len(df['Dígito_Verificador_Válido'].unique()) > 1:
    print('Existem CNPJs com dígito verificador inválido. Favor checar.')
    print(f'O CNPJ com dígito verificador inválido é {list(df["CNPJ"].unique())}')
    sys.exit()

In [73]:
def recalcular_digito_verificador(nfe_completa):
    # Verifica se a NF-e com o dígito verificador tem o tamanho correto
    if pd.isnull(nfe_completa) or len(nfe_completa) != 44:
        raise ValueError(f"Valor inválido na coluna 'CHV_DOC': {nfe_completa}")

    # Remove o último caractere (dígito verificador)
    nfe_sem_dv = nfe_completa[:-1]

    # Restante da lógica de cálculo do dígito verificador
    soma = 0
    peso = 2
    nfe_invertida = nfe_sem_dv[::-1]

    for digito in nfe_invertida:
        soma += int(digito) * peso
        peso += 1
        if peso > 9:
            peso = 2

    resto = soma % 11
    dv = 0 if resto == 0 or resto == 1 else 11 - resto

    return dv

In [75]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
    df_0000_['IE'] = df_0000_['IE'].str.replace('-', '')
    df_0000_ = df_0000_.drop('LOJA', axis=1)
    if df_0000_.shape[0] != 1:
        print(f'Erro no 0000 da loja {loja} para a data {date}. Favor verificar')
        sys.exit()
        
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final_new[df_0150_final_new['DATA'] == date]
    if df_0150_.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_['IE'] = df_0150_['IE'].str.replace('-', '')
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='cod novo', keep='first', inplace=True)
    for cod in list(df_0200['cod novo'].unique()):
        if df_0200[df_0200['cod novo'] == cod].shape[0] > 1:
            mensagem = 'ATENÇÃO. EXISTE CODIGO DE PRODUTO DUPLICADO NO 0200.'
            print(mensagem)
            print(f'O código duplicado é {cod}')
            sys.exit()
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['cod novo'].unique()
    arquivo_1050['cod novo'] = arquivo_1050['cod novo'].astype(str)
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['cod novo'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['cod novo','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
    df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'cod novo', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    if df_1100_[df_1100_['VLR_CONFR_PCAT'] < 0.01].shape[0] > 0:
        mensagem = 'ATNEÇÂO. EXISTE VALOR DE CONFRONTO IGUAL A 0.'
        print(mensagem)
        sys.exit()
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]
            
    cnpj_loja = df_0000[df_0000['LOJA'] == f'Loja {loja}']['CNPJ'].values[0]
    df_1100_['CHV_DOC'] = np.where(df_1100_['FONTE'].isin(['entrada_oprprp', 'saida_nfe', 'dfe']),
                           df_1100_['CHV_DOC'].str.slice(0,6) + cnpj_loja + df_1100_['CHV_DOC'].str.slice(20,44),
                           df_1100_['CHV_DOC'])
    df_1100_['DV_NFe'] = df_1100_['CHV_DOC'].apply(recalcular_digito_verificador)
    df_1100_['CHV_DOC'] = df_1100_['CHV_DOC'].str.slice(0,-1) + df_1100_['DV_NFe'].astype(str)
    
    df_1100_['CNPJ'] = df_1100_['CHV_DOC'].str.slice(6,20)
    for cnpj in df_1100_['CNPJ'].unique():
        if df_0150_[df_0150_['CNPJ'] == cnpj].shape[0] == 0:
            mensagem = 'ATENÇÃO. EXISTE PARTICIPANTE NO 1100 QUE NÃO ESTÁ NO 0150.'
            print(mensagem)
            print(f'O participante faltante no 0150 tem CNPJ {cnpj}, para o mês {date}')
            sys.exit()
    
    
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0000_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0150_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0200_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1050_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1100_{date}_{loja}_v1.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\*{date}_{loja}_v1.txt")
    
    with open(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\cat_Loja_{loja}_{date}_v1.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3412921052.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3412921052.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['cod novo'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3412921052.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_34652\3412921052.py:48: SettingWithCopyWarning: 
A value is trying to be set

ValueError: Valor inválido na coluna 'CHV_DOC': 351801nan550020000178621133143055